# Set up:
1) Modules
2) Credentials & keys for Trello and Google Sheets
3) Load Trello board
4) Configure pandas output

In [1]:
# Set up modules
import pandas as pd
import sys
import requests
from xlsxwriter.utility import xl_rowcol_to_cell
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf
import trello_generic as tg
import sqlite3
from google.cloud import bigquery # To run BQ statements
import re
# Set up SQL DB
conn = sqlite3.connect('SQL_connection1.db') #Create a connection object

# Set up credentials for Trello and Google 

# Google Sheets Credentials
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/') # Return logged-in credentials

# General setup and credentials: Trello
from trello import TrelloClient
trelloUserCreds=tg.readTrelloCredsFromFile(r'/home/jupyter/reusable_code/trellocreds.pickle')
mykey,mysecret,mytoken=trelloUserCreds

client = TrelloClient(api_key=mykey,api_secret=mysecret,token=mytoken)

# Return Trello board, client and other credentials objects. "myboard_creds" is a tuple of items which can be unpacked
# to cover off all of the various levels you might need access at
dataBoard,dataBoard_id,dataBoard_creds=tg.Return_board_by_name(mykey,mysecret,mytoken,"Data 2021")
researchBoard,researchBoard_id,researchBoard_creds=tg.Return_board_by_name(mykey,mysecret,mytoken,"Research 2021")
#oldBoard,oldBoard_id,oldBoard_creds=tg.Return_board_by_name(mykey,mysecret,mytoken,"Insights & Data")
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

# Read Trello board into a DataFrame

In [2]:
dataCard_df,dataCard_list=tg.cards_to_dataframe(dataBoard_creds,checklist_options=None\
                           ,labels_as_binary_flags=True, label_colours=True,comment_names=False,get_attachments=True #)
                                               ,card_number_cutoff=10, lists_to_exclude=['Template(s)','Ideation','Completed','No longer required'])

In [3]:
dataCard_list

# Turn "New Insight Brief(s)" into proper cards

In [ ]:
def monthToNum(shortMonth):
    return {
            'Jan' : '01',
            'Feb' : '02',
            'Mar' : '03',
            'Apr' : '04',
            'May' : '05',
            'Jun' : '06',
            'Jul' : '07',
            'Aug' : '08',
            'Sep' : '09', 
            'Oct' : '10',
            'Nov' : '11',
            'Dec' : '12'
    }[shortMonth]


def Process_New_Insight_Briefs():
    MonthLookup={}
    
    New_briefs=[i for i in dataCard_list if i['Name']=='New Insight Brief' and i['List'] in ['New Projects (need prioritisation)','Backlog', 'Prioritised','In Progress']]
    
    for i in New_briefs:
            CardInfo=re.split('Sent via Google Form Notifications',i['Description'])[0] # Take the bit before the generic email signature
            print(CardInfo)
            
            #############################
            # Reset card name
            #############################
            i['Card Object'].set_name(re.search('\*Project Name\*: \*(.*)\*',CardInfo)[1])

            #############################
            # Set Due Date            
            #############################
            DeadlineInfo=re.search('\*Needed by: \*(.*)',CardInfo)[1]
            
            try:
                DueDate=datetime.strptime(DeadlineInfo[:12], '%b %d, %Y')
                i['Card Object'].set_due(DueDate)
            except:
                pass
            
            #############################
            # Set labels for team        
            #############################
            try:
                Team=re.search('\*Submitted by\*: .* in the (.+) team',CardInfo)[1]
                try:
                    LabelObject=[i for i in labellist if i.name==Team][0]
                    i['Card Object'].add_label(LabelObject)
                except:
                    print('Could not add label for team: "{}"'.format(Team))
            except:
                pass
          
                
            #############################
            # Set labels for priorities/ projects            
            #############################
            projects=re.findall('\*Supports projects\*: (.*)',CardInfo)
            if len(projects)>0:
                projectList=re.split('\n',projects[0])
                for project in projectList:
                    try:
                        LabelObject=[i for i in labellist if i.name==project][0]
                        i['Card Object'].add_label(LabelObject)
                    except:
                        print('Could not add label for project: "{}"'.format(project))
    
            #############################
            # Set custom field (type)
            #############################
            WorkType=re.search('\*(.*) brief\*',CardInfo)[1]
            if WorkType=='No idea- you tell me':            # If unknown leave blank
                pass
            else:
                try:
                    tg.Update_custom_field(dataBoard_creds,i['Trello ID'],'Type',WorkType)
                except:
                    print('Could not update Type field with value: {}'.format(WorkType))
            
            #############################
            # Set custom field (Blocker) 
            try:
                Blockerinfo= re.search('\*Blocker: \*(.*)? which should be resolved by',CardInfo)[1]
                tg.Update_custom_field(dataBoard_creds,i['Trello ID'],'Blockers/ Dependencies 1',Blockerinfo)
            except:
                pass
            
            # Set custom field (Blocker date) 
            try:
                Blockerdatetext= re.search('\*Blocker: \*.* which should be resolved by (.+)',CardInfo)[1]
                BlockerDate= '-'.join([Blockerdatetext[8:12],monthToNum(Blockerdatetext[0:3]),Blockerdatetext[4:6]])   
                tg.Update_custom_field(dataBoard_creds,i['Trello ID'],'Blocker 1 Due Date',BlockerDate)
            except:
                pass
            
            # Redo description            
            NewDesc=re.split('Project Detail',CardInfo)[1]+'\n Submitted By: '\
            +re.search('\*Submitted by\*: (.*)@...',CardInfo)[1].replace('.',' ')\
            +'\nRequired by: '+DeadlineInfo
            i['Card Object'].set_description(NewDesc)
            
            # If research move to research board
            if WorkType=='Research':
                tg.MoveCard(i['Card Object'],'5fe35ef42dd5616a3e37bc12',mykey,mytoken,boardid=researchBoard_id)
    
    return New_briefs
   

In [ ]:
briefs=Process_New_Insight_Briefs()

In [ ]:
# Reimport
dataCard_df,dataCard_list=tg.cards_to_dataframe(dataBoard_creds,checklist_options=None\
                           ,labels_as_binary_flags=True, label_colours=False,comment_names=False,get_attachments=True #)
                                               ,card_number_cutoff=10000)

## Copy to BigQuery

In [12]:
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/')
bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds)
dataset=bq.dataset('britbox_sandbox')
table_ref = dataset.table("SW_Data_Workstack")

In [13]:
# Create a copy for amending then loading into BQ
df_for_bq=dataCard_df.copy()

# Most of the columns are "object" type, which holds mixed types. Explicitly make dates as such else it'll break load as it expects a "bytes" type then finds a datetime
df_for_bq['Due Date'] = pd.to_datetime(df_for_bq['Due Date'].astype(str))
df_for_bq['Card Created Date'] = pd.to_datetime(df_for_bq['Card Created Date'].astype(str))
df_for_bq['Hard Deadline'] = pd.to_datetime(df_for_bq['Hard Deadline'].astype(str))
df_for_bq['Blocker 1 Due Date'] = pd.to_datetime(df_for_bq['Blocker 1 Due Date'].astype(str))

# Struggles to identify and load array of STRUCTs with different datatypes, so just don't for now

df_for_bq=df_for_bq.drop(columns=['listMovementHistory', 'listMovementSummary'])


# Remove characters that you can't have in a BQ variable name
newcol_names={x:x.replace(" ", "_").replace("/","").replace("?","").replace("-","").replace("&","") for x in df_for_bq.columns}
df_for_bq=df_for_bq.rename(columns=newcol_names)

#Remove blank column names which might arise from blank labels on the board
df_col=[i for i in df_for_bq.columns if len(i)>0] 
df_for_bq=df_for_bq[df_col]

In [14]:
df_for_bq.head()

In [15]:
try:
    bq.delete_table(table_ref)
except:
    pass
job = bq.load_table_from_dataframe(df_for_bq, table_ref)

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

In [ ]:
df_for_bq.columns

# Tidy up DataFrame to include only the records and columns of interest

In [ ]:
query="""
create or replace table `itv-bde-analytics-dev.britbox_sandbox.SW_Data_Workstack1` as
with x  as (select 
Name
,Description
,List
,Due_Date
,Trello_ID
,Trello_URL
,Card_Created_Date
,Comments
,Trello_attachments
,Other_attachments
,Blocker_1_Due_Date
,Supported_by
,Ad_hoc
,Assigned_to
,Blockers_Dependencies_1
,Type
,Subtype
,Blockers_Dependencies_2
,Project_Brief_Location
,Paused_or_Blocked
,Mark_for_Deletion
,Hard_Deadline
,EPIC
,isEPIC
,split(Labels,'|') as labels1
from `itv-bde-analytics-dev.britbox_sandbox.SW_Data_Workstack`
where list not in ('Template(s)','No longer required')
)

select x.* except (labels1)
, array_agg(case when trim(split(labels2,':')[safe_offset(1)])='green' then 
trim(split(labels2,':')[safe_offset(0)]) end ignore nulls) as Teams
, array_agg(case when trim(split(labels2,':')[safe_offset(1)])='yellow' then 
trim(split(labels2,':')[safe_offset(0)]) end ignore nulls) as Priorities
, array_agg(case when trim(split(labels2,':')[safe_offset(1)])='blue' then 
trim(split(labels2,':')[safe_offset(0)]) end ignore nulls) as TeamObjectives
from x
cross join unnest (labels1) as labels2
group by 1,2,3,4,5,6,7,8,9,10
,11,12,13,14,15,16,17,18,19,20,21,22,23,24
;"""
df = bq.query(query).to_dataframe()

### Post-prioritisation session admin

##### Move all cards from New to Backog

In [ ]:

from_list=[i for i in dataBoard.list_lists() if i.name=='New Projects (need prioritisation)'][0]
to_list=[i for i in dataBoard.list_lists() if i.name=='Backlog'][0]
from_list.move_all_cards(to_list)   

##### Move all cards from Recently Completed to Completed

In [ ]:

from_list=[i for i in dataBoard.list_lists() if i.name=='Recently Completed'][0]
to_list=[i for i in dataBoard.list_lists() if i.name=='Completed'][0]
from_list.move_all_cards(to_list)   